<a href="https://colab.research.google.com/github/ashik950/Automation/blob/main/ParallelGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install paddlepaddle-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install paddleocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 93.6 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=ba6c0b25243a40caa7a6197fc35b290c38122e05dca8a2e594fb6f657cc5bc68
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [ ]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
import os
import time
import multiprocessing
from paddleocr import PaddleOCR
import torch

# Configuration for PaddleOCR
config = {
    "lang": "en",                              # Language setting (e.g., "en" for English)
    "use_gpu": torch.cuda.is_available(),      # Use GPU if available
    "det_model": "ch_ppocr_mobile_v2.0_det",   # Detection model
    "rec_model": "ch_ppocr_mobile_v2.0_rec",   # Recognition model
    'det_db_thresh': 0.3,                      # Lower threshold for binarization
    'det_char_conf': 0.3,                      # Lower minimum confidence level for character recognition
    'cls_conf_thresh': 0.5,
    "use_angle_cls": True,
    "use_space_char": True,
    "text_layout": True,
    "det_limit_type": "min",
    "max_text_length": 5000,                   # Maximum length of extracted text (experimental)
}

def initialize_ocr():
    global ocr
    ocr = PaddleOCR(**config)

def process_pdf(args):
    pdf_file, output_folder, results = args
    start_time = time.time()  # Record the start time
    try:
        results_from_ocr = ocr.ocr(pdf_file, cls=True)
        for i, page in enumerate(results_from_ocr):
            page_text = ""
            for line in page:
                text = line[1][0]  # Access the text content of the line
                page_text += f"{text}\n"

            output_file = os.path.join(output_folder, os.path.splitext(os.path.basename(pdf_file))[0] + f'_page_{i+1}.txt')

            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(page_text)

        end_time = time.time()  # Record the end time
        total_time = end_time - start_time
        result = f"Processed {pdf_file} in {total_time:.2f} seconds"
        results.append(result)
    except Exception as e:
        result = f"Error processing {pdf_file}: {e}"
        results.append(result)

def process_files(pdf_files, output_folder):
    print(f"Starting processing of {len(pdf_files)} files in {output_folder}")
    start_time = time.time()  # Record the start time for the whole processing
    manager = multiprocessing.Manager()
    results = manager.list()  # Shared list to store results

    num_processes = multiprocessing.cpu_count()  # Adjust this to the number of CPU cores you want to use
    with multiprocessing.Pool(num_processes, initializer=initialize_ocr) as pool:
        pool.map(process_pdf, [(pdf_file, output_folder, results) for pdf_file in pdf_files])

    end_time = time.time()  # Record the end time for the whole processing
    total_time = end_time - start_time
    print(f"Finished processing of {len(pdf_files)} files in {output_folder}. Total time taken: {total_time:.2f} seconds")

    # Print the results after all files have been processed
    for result in results:
        print(result)

# Example usage
if __name__ == "__main__":
    pdf_folder = "/content/sample_data/Input"
    output_folder = "/content/sample_data/Output"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.lower().endswith('.pdf')]
    process_files(pdf_files, output_folder)


FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/Input'

In [ ]:
import os
import time
import multiprocessing
from paddleocr import PaddleOCR
import torch

# Configuration for PaddleOCR
config = {
    "lang": "en",                              # Language setting (e.g., "en" for English)
    "use_gpu": torch.cuda.is_available(),      # Use GPU if available
    "det_model": "ch_ppocr_mobile_v2.0_det",   # Detection model
    "rec_model": "ch_ppocr_mobile_v2.0_rec",   # Recognition model
    'det_db_thresh': 0.3,                      # Lower threshold for binarization
    'det_char_conf': 0.3,                      # Lower minimum confidence level for character recognition
    'cls_conf_thresh': 0.5,
    "use_angle_cls": True,
    "use_space_char": True,
    "text_layout": True,
    "det_limit_type": "min",
    "max_text_length": 5000,                   # Maximum length of extracted text (experimental)
}

lock = multiprocessing.Lock()

def initialize_ocr(lock):
    with lock:
        global ocr
        ocr = PaddleOCR(**config)

def process_pdf(args):
    pdf_file, output_folder, results = args
    start_time = time.time()  # Record the start time
    try:
        results_from_ocr = ocr.ocr(pdf_file, cls=True)
        for i, page in enumerate(results_from_ocr):
            page_text = ""
            for line in page:
                text = line[1][0]  # Access the text content of the line
                page_text += f"{text}\n"

            output_file = os.path.join(output_folder, os.path.splitext(os.path.basename(pdf_file))[0] + f'_page_{i+1}.txt')

            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(page_text)

        end_time = time.time()  # Record the end time
        total_time = end_time - start_time
        result = f"Processed {pdf_file} in {total_time:.2f} seconds"
        results.append(result)
    except Exception as e:
        result = f"Error processing {pdf_file}: {e}"
        results.append(result)

def process_files(pdf_files, output_folder):
    print(f"Starting processing of {len(pdf_files)} files in {output_folder}")
    start_time = time.time()  # Record the start time for the whole processing
    manager = multiprocessing.Manager()
    results = manager.list()  # Shared list to store results

    num_processes = multiprocessing.cpu_count()  # Adjust this to the number of CPU cores you want to use
    with multiprocessing.Pool(num_processes, initializer=initialize_ocr, initargs=(lock,)) as pool:
        pool.map(process_pdf, [(pdf_file, output_folder, results) for pdf_file in pdf_files])

    end_time = time.time()  # Record the end time for the whole processing
    total_time = end_time - start_time
    print(f"Finished processing of {len(pdf_files)} files in {output_folder}. Total time taken: {total_time:.2f} seconds")

    # Print the results after all files have been processed
    for result in results:
        print(result)

# Example usage
if __name__ == "__main__":
    pdf_folder = "/content/sample_data/Input"
    output_folder = "/content/sample_data/Output"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.lower().endswith('.pdf')]

    # Pre-download models
    ocr = PaddleOCR(**config)  # Download models before multiprocessing
    process_files(pdf_files, output_folder)


In [ ]:
!pip list

In [ ]:
# Install PaddlePaddle with GPU support
!pip install paddlepaddle-gpu==2.5.0.post112 -f https://www.paddlepaddle.org.cn/whl/stable.html

# Install PaddleOCR
!pip install paddleocr

# Restart runtime (manually or using the command below)
# After running this cell, please manually restart the runtime or uncomment the next line to automatically restart.
# import os
# os._exit(00)


In [ ]:
!pip install paddlehub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.9/213.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.6/318.6 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.7/128.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━

In [ ]:
import paddlehub as hub

# Load the ERNIE model
model = hub.Module(name='ernie')

# Define the text data
texts = ["I love programming.", "The weather is terrible today."]

# Define the label map
label_map = {0: 'negative', 1: 'positive'}

# Get predictions
results = model.predict(texts)

# Print the results
for text, result in zip(texts, results):
    print(f'Text: {text}\nPrediction: {label_map[result]}\n')


In [ ]:
!pip install paddlepaddle-gpu==2.0.0

ERROR: Could not find a version that satisfies the requirement paddlepaddle-gpu==2.0.0 (from versions: 2.3.0, 2.3.1, 2.3.2, 2.4.0rc0, 2.4.0, 2.4.1, 2.4.2, 2.5.0rc0, 2.5.0rc1, 2.5.0, 2.5.1, 2.5.2, 2.6.0, 2.6.1)
ERROR: No matching distribution found for paddlepaddle-gpu==2.0.0


In [ ]:
import os
import time
import multiprocessing
from paddleocr import PaddleOCR
import torch
import paddlehub as hub


# Configuration for PaddleOCR
config = {
    "lang": "en",                              # Language setting (e.g., "en" for English)
    "use_gpu": torch.cuda.is_available(),      # Use GPU if available
    "det_model": "ch_ppocr_mobile_v2.0_det",   # Detection model
    "rec_model": "ch_ppocr_mobile_v2.0_rec",   # Recognition model
    'det_db_thresh': 0.3,                      # Lower threshold for binarization
    'det_char_conf': 0.3,                      # Lower minimum confidence level for character recognition
    'cls_conf_thresh': 0.5,
    "use_angle_cls": True,
    "use_space_char": True,
    "text_layout": True,
    "det_limit_type": "min",
    "max_text_length": 5000,                   # Maximum length of extracted text (experimental)
}

def initialize_ocr():
    global ocr
    ocr = PaddleOCR(**config)

def process_pdf(args):
    pdf_file, output_folder, results = args
    start_time = time.time()  # Record the start time
    try:
        results_from_ocr = ocr.ocr(pdf_file, cls=True)
        for i, page in enumerate(results_from_ocr):
            page_text = ""
            for line in page:
                text = line[1][0]  # Access the text content of the line
                page_text += f"{text}\n"

            output_file = os.path.join(output_folder, os.path.splitext(os.path.basename(pdf_file))[0] + f'_page_{i+1}.txt')

            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(page_text)

        end_time = time.time()  # Record the end time
        total_time = end_time - start_time
        result = f"Processed {pdf_file} in {total_time:.2f} seconds"
        results.append(result)
    except Exception as e:
        result = f"Error processing {pdf_file}: {e}"
        results.append(result)

def process_files(pdf_files, output_folder):
    print(f"Starting processing of {len(pdf_files)} files in {output_folder}")
    start_time = time.time()  # Record the start time for the whole processing
    manager = multiprocessing.Manager()
    results = manager.list()  # Shared list to store results

    num_processes = multiprocessing.cpu_count()  # Adjust this to the number of CPU cores you want to use
    with multiprocessing.Pool(num_processes, initializer=initialize_ocr) as pool:
        pool.map(process_pdf, [(pdf_file, output_folder, results) for pdf_file in pdf_files])

    end_time = time.time()  # Record the end time for the whole processing
    total_time = end_time - start_time
    print(f"Finished processing of {len(pdf_files)} files in {output_folder}. Total time taken: {total_time:.2f} seconds")

    # Print the results after all files have been processed
    for result in results:
        print(result)

# Example usage
if __name__ == "__main__":
    pdf_folder = "/content/Input"
    output_folder = "/content/output"

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.lower().endswith('.pdf')]
    process_files(pdf_files, output_folder)


Starting processing of 2 files in /content/output
[2024/08/05 10:51:18] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='min', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48

In [ ]:
import sys
print(sys.path)


['/usr/local/lib/python3.10/dist-packages/paddleocr', '/usr/local/lib/python3.10/dist-packages/paddleocr', '/usr/local/lib/python3.10/dist-packages/paddleocr/ppstructure', '/usr/local/lib/python3.10/dist-packages/paddleocr', '/usr/local/lib/python3.10/dist-packages/paddleocr', '/usr/local/lib/python3.10/dist-packages/paddleocr', '/usr/local/lib/python3.10/dist-packages/paddleocr', '/usr/local/lib/python3.10/dist-packages/paddleocr', '/usr/local/lib/python3.10/dist-packages/paddleocr', '/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor', '/root/.ipython', '/usr/local/lib/python3.10/dist-packages/paddleocr/', '/usr/local/lib/python3.10/dist-packages/paddleocr/tools/infer', '/usr/local/lib/python3.10/dist-packages/paddleocr/tools/

In [ ]:
!pip install paddlepaddle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 MB 8.5 MB/s eta 0:00:00


In [ ]:
!pip install paddlepaddle paddleocr

In [ ]:
!pip install paddlehub==2.4.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.9/213.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.6/318.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.7/128.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━

In [ ]:
!pip show paddlepaddle paddleocr paddlehub

Name: paddlepaddle
Version: 2.6.1
Summary: Parallel Distributed Deep Learning
Home-page: https://www.paddlepaddle.org.cn/
Author: 
Author-email: Paddle-better@baidu.com
License: Apache Software License
Location: /usr/local/lib/python3.10/dist-packages
Requires: astor, decorator, httpx, numpy, opt-einsum, Pillow, protobuf
Required-by: 
---
Name: paddleocr
Version: 2.8.1
Summary: Awesome OCR toolkits based on PaddlePaddle(8.6M ultra-lightweight pre-trained model, support training and deployment among server, mobile, embedded and IoT devices)
Home-page: https://github.com/PaddlePaddle/PaddleOCR
Author: 
Author-email: PaddlePaddle <Paddle-better@baidu.com>
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: beautifulsoup4, cython, fire, fonttools, imgaug, lmdb, numpy, opencv-contrib-python, opencv-python, Pillow, pyclipper, python-docx, pyyaml, rapidfuzz, requests, scikit-image, shapely, tqdm
Required-by: 
---
Name: paddlehub
Version: 2.4.0
Summary: A to

In [ ]:
# Install PaddlePaddle (GPU version)
!pip install paddlepaddle-gpu==2.0.0.post101 -f https://www.paddlepaddle.org.cn/whl/stable.html

# Install PaddleHub
!pip install paddlehub==2.0.0

# Verify the installations
import paddle
import paddlehub as hub

print(paddle.__version__)  # Should be 2.0.0
print(hub.__version__)     # Should be 2.0.0

# Load the ERNIE model
ernie_model = hub.Module(name='ernie')


Looking in links: https://www.paddlepaddle.org.cn/whl/stable.html
ERROR: Could not find a version that satisfies the requirement paddlepaddle-gpu==2.0.0.post101 (from versions: 2.3.0, 2.3.1, 2.3.2, 2.4.0rc0, 2.4.0, 2.4.1, 2.4.2, 2.5.0rc0, 2.5.0rc1, 2.5.0, 2.5.1, 2.5.2, 2.6.0, 2.6.1)
ERROR: No matching distribution found for paddlepaddle-gpu==2.0.0.post101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.6/191.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: paddlehub
    Found existing installation: paddlehub 2.4.0
    Uninstalling paddlehub-2.4.0:
      Successfully uninstalled paddlehub-2.4.0


2.6.1
2.4.0


EnvironmentMismatchError: [31mernie[39m cannot be installed because some conditions are not met:
+---------------+----------+-----------------------------------+-----------------------------------+
|[36m     Name      [39m|[36m Version  [39m|[36m   PaddlePaddle Version Required   [39m|[36m    PaddleHub Version Required     [39m|
+---------------+----------+-----------------------------------+-----------------------------------+
|     ernie     |  1.0.0   |                Any                |                Any                |
+---------------+----------+-----------------------------------+-----------------------------------+
|     ernie     |  1.0.1   |                Any                |                Any                |
+---------------+----------+-----------------------------------+-----------------------------------+
|     ernie     |  1.0.2   |                Any                |                Any                |
+---------------+----------+-------------------------

In [ ]:
!pip list


Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.32.1
aiofiles                         23.2.1
aiohttp                          3.9.5
aiosignal                        1.3.1
aistudio-sdk                     0.2.5
alabaster                        0.7.16
albucore                         0.0.12
albumentations                   1.4.12
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.18.0
asn1crypto                       1.5.1
astor                            0.8.1
astropy                          6.1.2
astropy-iers-data                0.2024.7.29.0.32.7
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                 

In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 101.5 MB/s eta 0:00:00


In [ ]:
import os
import time
import multiprocessing
from paddleocr import PaddleOCR
import torch
import paddlehub as hub
import fitz  # PyMuPDF

# Configuration for PaddleOCR
config = {
    "lang": "en",                              # Language setting (e.g., "en" for English)
    "use_gpu": torch.cuda.is_available(),      # Use GPU if available
    "det_model": "ch_ppocr_mobile_v2.0_det",   # Detection model
    "rec_model": "ch_ppocr_mobile_v2.0_rec",   # Recognition model
    'det_db_thresh': 0.3,                      # Lower threshold for binarization
    'det_char_conf': 0.3,                      # Lower minimum confidence level for character recognition
    'cls_conf_thresh': 0.5,
    "use_angle_cls": True,
    "use_space_char": True,
    "text_layout": True,
    "det_limit_type": "min",
    "max_text_length": 5000,                   # Maximum length of extracted text (experimental)
}

def initialize_ocr():
    global ocr
    ocr = PaddleOCR(**config)

def process_pdf(args):
    pdf_file, output_folder, results = args
    start_time = time.time()  # Record the start time
    try:
        results_from_ocr = ocr.ocr(pdf_file, cls=True)
        for i, page in enumerate(results_from_ocr):
            page_text = ""
            for line in page:
                text = line[1][0]  # Access the text content of the line
                page_text += f"{text}\n"

            output_file = os.path.join(output_folder, os.path.splitext(os.path.basename(pdf_file))[0] + f'_page_{i+1}.txt')

            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(page_text)

        end_time = time.time()  # Record the end time
        total_time = end_time - start_time
        result = f"Processed {pdf_file} in {total_time:.2f} seconds"
        results.append(result)
    except Exception as e:
        result = f"Error processing {pdf_file}: {e}"
        results.append(result)

def process_files(pdf_files, output_folder):
    print(f"Starting processing of {len(pdf_files)} files in {output_folder}")
    start_time = time.time()  # Record the start time for the whole processing
    manager = multiprocessing.Manager()
    results = manager.list()  # Shared list to store results

    num_processes = multiprocessing.cpu_count()  # Adjust this to the number of CPU cores you want to use
    with multiprocessing.Pool(num_processes, initializer=initialize_ocr) as pool:
        pool.map(process_pdf, [(pdf_file, output_folder, results) for pdf_file in pdf_files])

    end_time = time.time()  # Record the end time for the whole processing
    total_time = end_time - start_time
    print(f"Finished processing of {len(pdf_files)} files in {output_folder}. Total time taken: {total_time:.2f} seconds")

    # Print the results after all files have been processed
    for result in results:
        print(result)

# Example usage
if __name__ == "__main__":
    pdf_folder = "/content/Input"
    output_folder = "/content/output"

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.lower().endswith('.pdf')]
    process_files(pdf_files, output_folder)


Starting processing of 2 files in /content/output
[2024/08/05 12:12:54] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='min', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48